# Descubra quem fez o ENEM 2016 apenas para treino

### Neste desafio deverá descobrir quais estudantes estão fazendo a prova apenas para treino.

In [1]:
# Importanto bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Leitura dos dados

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# Visualizando dados de treino

train.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,...,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,...,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,...,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,...,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,...,NaN,A,A,A,A,A,A,A,A,A


In [4]:
# Visualizando dados de teste

test.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,43,RS,19,F,1,1,1,3,1,...,NaN,NaN,E,E,H,B,B,C,D,A
1,177f281c68fa032aedbd842a745da68490926cd2,15,PA,24,M,3,2,1,4,1,...,NaN,NaN,B,C,B,A,A,C,D,A
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,29,BA,16,F,2,1,3,0,1,...,NaN,NaN,E,F,G,B,B,A,NaN,D
3,5c356d810fa57671402502cd0933e5601a2ebf1e,41,PR,17,F,1,1,2,0,2,...,40.0,480.0,E,E,E,C,B,B,C,A
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,53,DF,19,F,1,1,1,1,1,...,80.0,720.0,E,E,E,B,B,B,D,A


In [5]:
# Dimensões dos dados de treino e teste

print(f'Shape dados de treino:', train.shape)
print(f'Shape dados de teste:', test.shape)

Shape dados de treino: (13730, 167)
Shape dados de teste: (4570, 43)


In [6]:
# Colunas que estão em ambos datasets

col = [x for x in train.columns if x in (test.columns)]
new_train = train[col]

In [7]:
# Dimensões dos dados de treino e teste

print(f'Shape dados de treino:', new_train.shape)
print(f'Shape dados de teste:', test.shape)

Shape dados de treino: (13730, 43)
Shape dados de teste: (4570, 43)


In [8]:
# Info dados de treino

new_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13730 entries, 0 to 13729
Data columns (total 43 columns):
NU_INSCRICAO              13730 non-null object
CO_UF_RESIDENCIA          13730 non-null int64
SG_UF_RESIDENCIA          13730 non-null object
NU_IDADE                  13730 non-null int64
TP_SEXO                   13730 non-null object
TP_COR_RACA               13730 non-null int64
TP_NACIONALIDADE          13730 non-null int64
TP_ST_CONCLUSAO           13730 non-null int64
TP_ANO_CONCLUIU           13730 non-null int64
TP_ESCOLA                 13730 non-null int64
TP_ENSINO                 4282 non-null float64
TP_DEPENDENCIA_ADM_ESC    4282 non-null float64
IN_BAIXA_VISAO            13730 non-null int64
IN_CEGUEIRA               13730 non-null int64
IN_SURDEZ                 13730 non-null int64
IN_DISLEXIA               13730 non-null int64
IN_DISCALCULIA            13730 non-null int64
IN_SABATISTA              13730 non-null int64
IN_GESTANTE               13730 non-null

In [9]:
# sumarização dos dados de treino

new_train.describe().round(2)

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,...,NU_NOTA_CH,NU_NOTA_LC,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
count,13730.00,13730.00,13730.00,13730.00,13730.00,13730.00,13730.00,4282.00,4282.00,13730.00,...,10341.00,10133.00,13730.00,10133.0,10133.00,10133.00,10133.00,10133.00,10133.00,10133.00
mean,30.74,21.59,2.19,1.05,1.78,2.15,1.35,1.18,2.26,0.00,...,529.66,516.47,0.59,1.1,119.81,111.92,106.82,113.12,77.37,529.05
std,9.86,7.04,1.01,0.26,0.88,3.22,0.56,0.57,0.70,0.04,...,73.73,68.69,0.49,0.7,29.85,33.74,34.34,33.48,43.07,154.29
min,11.00,13.00,0.00,0.00,1.00,0.00,1.00,1.00,1.00,0.00,...,0.00,0.00,0.00,1.0,0.00,0.00,0.00,0.00,0.00,0.00
25%,23.00,17.00,1.00,1.00,1.00,0.00,1.00,1.00,2.00,0.00,...,480.40,468.10,0.00,1.0,120.00,100.00,80.00,100.00,40.00,440.00
50%,31.00,19.00,3.00,1.00,2.00,0.00,1.00,1.00,2.00,0.00,...,532.00,520.90,1.00,1.0,120.00,120.00,120.00,120.00,80.00,540.00
75%,35.00,23.00,3.00,1.00,2.00,3.00,2.00,1.00,2.00,0.00,...,581.20,564.90,1.00,1.0,140.00,120.00,120.00,120.00,100.00,600.00
max,53.00,67.00,5.00,4.00,4.00,10.00,4.00,3.00,4.00,1.00,...,807.00,763.60,1.00,9.0,200.00,200.00,200.00,200.00,200.00,1000.00


In [10]:
# % de dados faltantes no treino

round(new_train.isna().mean()*100, 2)

NU_INSCRICAO               0.00
CO_UF_RESIDENCIA           0.00
SG_UF_RESIDENCIA           0.00
NU_IDADE                   0.00
TP_SEXO                    0.00
TP_COR_RACA                0.00
TP_NACIONALIDADE           0.00
TP_ST_CONCLUSAO            0.00
TP_ANO_CONCLUIU            0.00
TP_ESCOLA                  0.00
TP_ENSINO                 68.81
TP_DEPENDENCIA_ADM_ESC    68.81
IN_BAIXA_VISAO             0.00
IN_CEGUEIRA                0.00
IN_SURDEZ                  0.00
IN_DISLEXIA                0.00
IN_DISCALCULIA             0.00
IN_SABATISTA               0.00
IN_GESTANTE                0.00
IN_IDOSO                   0.00
TP_PRESENCA_CN             0.00
TP_PRESENCA_CH             0.00
TP_PRESENCA_LC             0.00
TP_PRESENCA_MT             0.00
NU_NOTA_CN                24.68
NU_NOTA_CH                24.68
NU_NOTA_LC                26.20
TP_LINGUA                  0.00
TP_STATUS_REDACAO         26.20
NU_NOTA_COMP1             26.20
NU_NOTA_COMP2             26.20
NU_NOTA_

In [11]:
# % de dados faltantes no test

round(test.isna().mean()*100, 2)

NU_INSCRICAO               0.00
CO_UF_RESIDENCIA           0.00
SG_UF_RESIDENCIA           0.00
NU_IDADE                   0.00
TP_SEXO                    0.00
TP_COR_RACA                0.00
TP_NACIONALIDADE           0.00
TP_ST_CONCLUSAO            0.00
TP_ANO_CONCLUIU            0.00
TP_ESCOLA                  0.00
TP_ENSINO                 68.80
TP_DEPENDENCIA_ADM_ESC    68.80
IN_BAIXA_VISAO             0.00
IN_CEGUEIRA                0.00
IN_SURDEZ                  0.00
IN_DISLEXIA                0.00
IN_DISCALCULIA             0.00
IN_SABATISTA               0.00
IN_GESTANTE                0.00
IN_IDOSO                   0.00
TP_PRESENCA_CN             0.00
TP_PRESENCA_CH             0.00
TP_PRESENCA_LC             0.00
TP_PRESENCA_MT             0.00
NU_NOTA_CN                24.33
NU_NOTA_CH                24.33
NU_NOTA_LC                25.60
TP_LINGUA                  0.00
TP_STATUS_REDACAO         25.60
NU_NOTA_COMP1             25.60
NU_NOTA_COMP2             25.60
NU_NOTA_

In [12]:
# Adicionando a coluna treineiro no new_train do train

column = train['IN_TREINEIRO']
new_train = pd.concat([new_train, column], axis = 1)

In [13]:
# Valores = 0 correspondem a não treineiros e valores = 1 correspondem a treineiros

new_train['IN_TREINEIRO'].value_counts()

0    11947
1     1783
Name: IN_TREINEIRO, dtype: int64

In [14]:
# Tratando valores nulos

new_train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [15]:
# Análise exploratória dos dados - Visualizando correlações entre variáveis
# Usarei a cosine similarity tendo em vista que a correlação corr do pandas compara somente
# as variáveis contínuas

# Variáveis categóricas
var_categ = ['CO_UF_RESIDENCIA', 'NU_IDADE','TP_SEXO' ,'TP_COR_RACA', 'TP_NACIONALIDADE','TP_ST_CONCLUSAO',
               'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DISLEXIA',
               'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE', 'IN_IDOSO', 'TP_PRESENCA_CN','TP_PRESENCA_CH',
               'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'Q001', 'Q002', 'Q006',
               'Q024','Q025','Q026','Q047']

In [16]:
# Transformo em variáveis categóricas
train_categ = new_train[var_categ].astype('category')

# Transformo variáveis object em categóricas númericas (visualizadas em new_train.info())
objetc_col = ['TP_SEXO', "Q001", "Q002", "Q006", "Q024", "Q025", "Q026", "Q047"]

train_categ[objetc_col] = train_categ[objetc_col].apply(lambda x: x.cat.codes)

In [17]:
# Aplicando a cosine similarity

x = train_categ.iloc[:, :-1].values
y = new_train['IN_TREINEIRO'].values

In [18]:
for row in x.T:
    result = cosine_similarity(x.T, y.reshape(1, -1))

In [19]:
result_cosine = pd.DataFrame(result, index = train_categ.iloc[:, :-1].columns)
result_cosine = result_cosine.sort_values(by = 0, ascending = False)
result_cosine

,0
TP_ST_CONCLUSAO,0.543712
TP_PRESENCA_CH,0.356627
TP_PRESENCA_CN,0.356627
Q002,0.354362
TP_PRESENCA_LC,0.353723
TP_PRESENCA_MT,0.353723
TP_NACIONALIDADE,0.346471
Q001,0.337425
CO_UF_RESIDENCIA,0.333175
TP_COR_RACA,0.331059


In [20]:
# Seleciono as correlações acima de 35%

cosine_top = result_cosine.iloc[:6]
cosine_top

,0
TP_ST_CONCLUSAO,0.543712
TP_PRESENCA_CH,0.356627
TP_PRESENCA_CN,0.356627
Q002,0.354362
TP_PRESENCA_LC,0.353723
TP_PRESENCA_MT,0.353723


In [21]:
optimize_train = train_categ[cosine_top.index]

optimize_train = pd.concat([optimize_train, new_train['IN_TREINEIRO']], axis = 1)
optimize_train

,TP_ST_CONCLUSAO,TP_PRESENCA_CH,TP_PRESENCA_CN,Q002,TP_PRESENCA_LC,TP_PRESENCA_MT,IN_TREINEIRO
0,1,1,1,3,1,1,0
1,2,1,1,0,1,1,0
2,3,0,0,3,0,0,0
3,1,0,0,4,0,0,0
4,1,0,0,3,0,0,0
...,...,...,...,...,...,...,...
13725,1,1,1,4,1,1,0
13726,1,1,1,0,1,1,0
13727,3,1,1,6,1,1,1
13728,4,1,1,3,1,1,0


In [22]:
# Salvo o 'NU_INSCRICAO' e retiro dos dados de teste

NU_INSCRICAO = test[['NU_INSCRICAO']]  # salvo

test.drop(columns = 'NU_INSCRICAO', axis = 1, inplace = True)

In [23]:
test.head()

,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,43,RS,19,F,1,1,1,3,1,0.0,...,0.0,0.0,E,E,H,B,B,C,D,A
1,15,PA,24,M,3,2,1,4,1,0.0,...,0.0,0.0,B,C,B,A,A,C,D,A
2,29,BA,16,F,2,1,3,0,1,0.0,...,0.0,0.0,E,F,G,B,B,A,0,D
3,41,PR,17,F,1,1,2,0,2,1.0,...,40.0,480.0,E,E,E,C,B,B,C,A
4,53,DF,19,F,1,1,1,1,1,0.0,...,80.0,720.0,E,E,E,B,B,B,D,A


In [24]:
# graficos

In [25]:
# feature eng

optimize_train = pd.get_dummies(optimize_train, columns = ['TP_ST_CONCLUSAO', "Q002"])
optimize_train

,TP_PRESENCA_CH,TP_PRESENCA_CN,TP_PRESENCA_LC,TP_PRESENCA_MT,IN_TREINEIRO,TP_ST_CONCLUSAO_1,TP_ST_CONCLUSAO_2,TP_ST_CONCLUSAO_3,TP_ST_CONCLUSAO_4,Q002_0,Q002_1,Q002_2,Q002_3,Q002_4,Q002_5,Q002_6,Q002_7
0,1,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0
1,1,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,1,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0
13726,1,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0
13727,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,1,0
13728,1,1,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0


In [26]:
# Let's model!

X = optimize_train.drop(columns = 'IN_TREINEIRO').astype(np.number)
y = optimize_train[['IN_TREINEIRO']]

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 45)

In [28]:
random_forest = RandomForestClassifier()

In [29]:
random_forest.fit(X_train, y_train)

RandomForestClassifier()

In [30]:
random_forest.predict(X_test)

array([0, 1, 0, ..., 0, 0, 0])

In [31]:
random_forest.score(X_test, y_test)

0.961034231609614

In [32]:
# Balanceando os dados

sm = SMOTE(random_state = 44)

X_balanced , y_balanced = sm.fit_resample(X_train, y_train)

In [33]:
random_forest_balanced = RandomForestClassifier().fit(X_balanced , y_balanced)

In [34]:
random_forest_balanced.predict(X_balanced)

array([0, 0, 0, ..., 1, 1, 1])

In [35]:
random_forest_balanced.score(X_balanced , y_balanced)

0.9778301886792453

In [36]:
pred = random_forest_balanced.predict(X_balanced)

In [37]:
test = test[cosine_top.index]
test = pd.get_dummies(test, columns = ['TP_ST_CONCLUSAO', "Q002"])

pred2 = random_forest_balanced.predict(test)

In [38]:
# Salvo o arquivo

NU_INSCRICAO['IN_TREINEIRO'] = pred2

NU_INSCRICAO.to_csv('answer.csv', index = False)

In [39]:
NU_INSCRICAO

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0
...,...,...
4565,361b7fcd8867119550fe2af5aa729ffad89a7cf5,0
4566,d8a0e4c9e29494cc9bba2422bd79333931475ee1,0
4567,3f1c3388244df8d6521e983a809292d9f3bca643,0
4568,1778e9c4cef591beb6b986d191d15ed05de816b0,0
